In [16]:
import pypgx.pg.rdbms.graph_server as graph_server

base_url = "https://localhost:7007"
username = "telfraud"
password = "WELcome123##"

instance = graph_server.get_instance(base_url, username, password)
session = instance.create_session("jupyter")
analyst = session.create_analyst()

print(session)

PgxSession(id: 66e7b5f8-8203-45df-9cf9-091348499440, name: jupyter)


In [126]:
statement = """
CREATE PROPERTY GRAPH "telfraud"
  VERTEX TABLES (
    telfraud.customer
      KEY (cst_id)
      LABEL customer
      PROPERTIES (cst_id, first_name, last_name)
  , telfraud.account
      KEY (acc_id)
      LABEL account
      PROPERTIES (acc_id, tel_number, is_suspect, is_victim)
  )
  EDGE TABLES (
    telfraud.transaction
      KEY (txn_id)
      SOURCE KEY(acc_id_src) REFERENCES account
      DESTINATION KEY(acc_id_dst) REFERENCES account
      LABEL called 
  , telfraud.account
      KEY (acc_id)
      SOURCE KEY(cst_id) REFERENCES customer
      DESTINATION KEY(acc_id) REFERENCES account
      LABEL owns
  )
"""

In [127]:
session.prepare_pgql(statement).execute()

False

In [124]:
graph = session.get_graph("telfraud")
graph

PgxGraph(name: telfraud, v: 180, e: 2884, directed: True, memory(Mb): 0)

In [129]:
graph.destroy()

In [134]:
statement = """
CREATE PROPERTY GRAPH "telfraud_call"
  VERTEX TABLES (
    telfraud.account
      KEY (acc_id)
      LABEL account
      PROPERTIES (acc_id, tel_number, is_suspect, is_victim)
  )
  EDGE TABLES (
    telfraud.transaction
      KEY (txn_id)
      SOURCE KEY(acc_id_src) REFERENCES account
      DESTINATION KEY(acc_id_dst) REFERENCES account
      LABEL called 
  )
"""

In [135]:
session.prepare_pgql(statement).execute()

False

In [136]:
graph = session.get_graph("telfraud_call")
graph

PgxGraph(name: telfraud_call, v: 100, e: 2787, directed: True, memory(Mb): 0)

In [137]:
graph.query_pgql("""
SELECT a.acc_id, a.tel_number, a.is_suspect, a.is_victim
FROM MATCH (a)
WHERE a.is_suspect = 1 OR a.is_victim = 1
LIMIT 15
""").print()

+-------------------------------------------------+
| acc_id | tel_number    | is_suspect | is_victim |
+-------------------------------------------------+
| 0      | 080-7890-1495 | 1          | 0         |
| 2      | 070-2500-2041 | 1          | 0         |
| 4      | 070-5969-2266 | 1          | 0         |
| 6      | 070-8946-4213 | 1          | 0         |
| 8      | 070-7068-5611 | 1          | 0         |
| 64     | 080-3045-8743 | 0          | 1         |
| 66     | 070-6111-6340 | 0          | 1         |
| 74     | 070-6230-2482 | 0          | 1         |
| 1      | 070-2033-4132 | 1          | 0         |
| 3      | 070-9916-2602 | 1          | 0         |
| 5      | 080-6874-9484 | 1          | 0         |
| 7      | 080-3471-5851 | 1          | 0         |
| 9      | 080-7116-4282 | 1          | 0         |
+-------------------------------------------------+



In [138]:
graph.query_pgql("""
SELECT a1.tel_number AS a1, a2.tel_number AS a2, c.datetime
FROM MATCH (a1)-[c:CALLED]->(a2)
LIMIT 5
""").print()

+-----------------------------------------------------+
| a1            | a2            | datetime            |
+-----------------------------------------------------+
| 080-7890-1495 | 070-4361-6313 | 2020-11-11T06:58:49 |
| 080-7890-1495 | 070-4361-6313 | 2020-11-15T01:41:55 |
| 080-7890-1495 | 070-4361-6313 | 2020-11-18T05:22:43 |
| 080-7890-1495 | 080-1956-1255 | 2020-11-17T19:09:53 |
| 080-7890-1495 | 080-1956-1255 | 2020-11-19T15:21:33 |
+-----------------------------------------------------+



In [139]:
analyst.in_degree_centrality(graph)

VertexProperty(name: in_degree, type: integer, graph: telfraud_call)

In [140]:
analyst.out_degree_centrality(graph)

VertexProperty(name: out_degree, type: integer, graph: telfraud_call)

In [141]:
graph.query_pgql("""
SELECT a.acc_id, a.tel_number, a.in_degree, a.out_degree, a.is_suspect
FROM MATCH (a)
WHERE in_degree < 5
LIMIT 100
""").print()

+--------------------------------------------------------------+
| acc_id | tel_number    | in_degree | out_degree | is_suspect |
+--------------------------------------------------------------+
| 0      | 080-7890-1495 | 1         | 40         | 1          |
| 2      | 070-2500-2041 | 0         | 9          | 1          |
| 4      | 070-5969-2266 | 0         | 20         | 1          |
| 6      | 070-8946-4213 | 1         | 50         | 1          |
| 8      | 070-7068-5611 | 0         | 16         | 1          |
| 1      | 070-2033-4132 | 0         | 20         | 1          |
| 3      | 070-9916-2602 | 0         | 20         | 1          |
| 5      | 080-6874-9484 | 1         | 47         | 1          |
| 7      | 080-3471-5851 | 0         | 29         | 1          |
| 9      | 080-7116-4282 | 0         | 38         | 1          |
+--------------------------------------------------------------+

